📘 Notebook Section 1

Document Ingestion & Chunking Pipeline (Arabic-Friendly)

This section ingests Arabic PDF documents, extracts and normalizes text, then splits it into overlapping semantic chunks suitable for embedding and retrieval in a RAG system.

In [ ]:
# Colab-ready robust PDF -> chunks pipeline (Arabic-friendly)
# التعليمات: ارفع ZIP إلى /content ثم شغّل الخلية.
!pip install PyPDF2 pdfplumber --quiet

import zipfile, os, re, time
from pathlib import Path
from PyPDF2 import PdfReader
import pdfplumber
import pandas as pd

# تعديل هذا المسار إذا رفعت الملف في مكان آخر
ZIP_PATH = Path('/content/100-day-kit-arabic_0.zip')
EXTRACT_DIR = Path('/content/extracted_pdfs')
OUT_CSV = Path('/content/processed_chunks.csv')

# إعداد
EXTRACT_DIR.mkdir(parents=True, exist_ok=True)

# 1) فك الضغط
print("Unzipping:", ZIP_PATH)
with zipfile.ZipFile(ZIP_PATH, 'r') as zf:
    zf.extractall(EXTRACT_DIR)
print("Extracted to:", EXTRACT_DIR)

# 2) جمع كل ملفات PDF
pdf_paths = sorted(EXTRACT_DIR.rglob('*.pdf'))
print("Found PDFs:", len(pdf_paths))

# 3) دوال استخراج سريعة وموثوقة
def extract_text_pypdf2(path):
    parts = []
    try:
        reader = PdfReader(str(path))
        for page in reader.pages:
            try:
                txt = page.extract_text()
                if txt:
                    parts.append(txt)
            except Exception:
                continue
    except Exception as e:
        # طباعة الخطأ لكن لا توقف التشغيل
        print(f"[PyPDF2 error] {path.name}: {e}")
    return "\n".join(parts).strip()

def extract_text_pdfplumber(path):
    parts = []
    try:
        with pdfplumber.open(path) as pdf:
            for page in pdf.pages:
                try:
                    txt = page.extract_text()
                    if txt:
                        parts.append(txt)
                except Exception:
                    continue
    except Exception as e:
        print(f"[pdfplumber error] {path.name}: {e}")
    return "\n".join(parts).strip()

def normalize_arabic_text(text):
    txt = text.replace('\r', '\n')
    txt = re.sub(r'\n\s+\n', '\n', txt) # إزالة الأسطر الفارغة المكررة
    txt = re.sub(r'[ \t]+', ' ', txt) # توحيد المسافات (بدل مسافتين تصبح واحدة)
    txt = re.sub(r'\n{2,}', '\n\n', txt)
    # حذف أرقام صفحات شائعة
    txt = re.sub(r'(?i)page\s*\d+|\d+\s*/\s*\d+|\bصفحة\b\s*\d+', '', txt) # إزالة كلمات مثل "Page 1" أو "صفحة 5" لأنها تشوش على البوت
    lines = [ln.strip() for ln in txt.splitlines()]
    lines = [ln for ln in lines if len(ln) > 1]
    return "\n".join(lines).strip()

# 4) تقسيم ذكي إلى مقاطع (chunking)
SENT_DELIMS = r'(?<=[\.\?\!\؟\!؛\n])\s+'
TARGET_CHARS = 1000
OVERLAP_CHARS = 200

# سننشئ CSV بشكل تراكمي: نكتب صفوف كل ملف فوراً لتفادي فقدان العمل إذا حدث خطأ لاحقاً
rows_written = 0
with open(OUT_CSV, 'w', encoding='utf-8', newline='') as fout:
    import csv
    writer = csv.DictWriter(fout, fieldnames=['id','source_pdf','chunk_index','text','char_len'])
    writer.writeheader()
    chunk_id = 0

    for p in pdf_paths:
        t0 = time.time()
        print(f"\nProcessing {p.name} ...", end=' ')
        # 1) حاول استخراج سريعا بـ PyPDF2
        raw = extract_text_pypdf2(p)
        # 2) إن كانت النتيجة ضعيفة، جرب pdfplumber
        if not raw or len(raw) < 50:
            raw = extract_text_pdfplumber(p)
        if not raw or len(raw) < 50:
            print("SKIPPED (no text)")
            continue
        norm = normalize_arabic_text(raw)
        # تقسيم إلى "جمل" ثم تجميع إلى chunks
        parts = re.split(SENT_DELIMS, norm)
        cur = ""
        chunk_idx = 0
        for part in parts:
            if not part or len(part.strip())==0:
                continue
            part = part.strip()
            if len(cur) + len(part) <= TARGET_CHARS or len(cur) == 0:
                cur = (cur + " " + part).strip() if cur else part
            else:
                writer.writerow({
                    'id': f"chunk_{chunk_id:06d}",
                    'source_pdf': str(p.name),
                    'chunk_index': chunk_idx,
                    'text': cur,
                    'char_len': len(cur)
                })
                rows_written += 1
                chunk_id += 1
                chunk_idx += 1
                # overlap
                overlap = cur[-OVERLAP_CHARS:] if OVERLAP_CHARS < len(cur) else cur
                cur = (overlap + " " + part).strip()
        if cur and len(cur.strip())>0:
            writer.writerow({
                'id': f"chunk_{chunk_id:06d}",
                'source_pdf': str(p.name),
                'chunk_index': chunk_idx,
                'text': cur,
                'char_len': len(cur)
            })
            rows_written += 1
            chunk_id += 1
        print(f"done in {time.time()-t0:.1f}s, chars={len(raw)} (rows_written={rows_written})")

print(f"\nAll done. CSV saved to: {OUT_CSV}  (total chunks: {rows_written})")


📘 Notebook Section 2

Embedding, Vector Store Construction & RAG Query Engine

This section converts text chunks into vector embeddings, stores them in a persistent vector database, and builds a Retrieval-Augmented Generation (RAG) pipeline for grounded question answering.

In [ ]:
# 1. تحديث وتثبيت المكتبات (تشمل الحل لتعارض الإصدارات)
!pip install -U langchain-google-genai langchain-community chromadb sentence-transformers langchain-chroma pandas==2.2.2 --quiet

import os
import pandas as pd
import shutil
import chromadb
from pathlib import Path
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

# --- الإعدادات الأساسية ---
os.environ["GEMINI_API_KEY"] = "-put your GEMINI_API_KEY here"
OUT_CSV = Path('/content/processed_chunks.csv')
CHROMA_DB_PATH = "/content/autism_db_fixed"
COLLECTION_NAME = "autism_collection"

# 2. تحميل نموذج التضمين (E5 المخصص للغة العربية)
print("⏳ جارٍ تحميل نموذج التضمين...")
embeddings = HuggingFaceEmbeddings(model_name='intfloat/multilingual-e5-large')

# 3. دالة بناء أو تحميل قاعدة البيانات
def get_vectorstore():
    # التحقق مما إذا كانت قاعدة البيانات موجودة مسبقاً
    if os.path.exists(CHROMA_DB_PATH) and os.listdir(CHROMA_DB_PATH):
        print("✅ تم العثور على قاعدة بيانات جاهزة، يتم تحميلها الآن...")
        return Chroma(
            persist_directory=CHROMA_DB_PATH,
            embedding_function=embeddings,
            collection_name=COLLECTION_NAME
        )

    # إذا لم تكن موجودة، نبدأ ببنائها بالدفعات (Batch Processing) لتوفير الذاكرة
    if not OUT_CSV.exists():
        raise FileNotFoundError(f"⚠️ ملف {OUT_CSV} غير موجود! يرجى تشغيل جزء استخراج البيانات أولاً.")

    print(f"🚀 قاعدة البيانات غير موجودة. بدء المعالجة من ملف CSV: {OUT_CSV}")
    os.makedirs(CHROMA_DB_PATH, exist_ok=True)

    # تهيئة عميل ChromaDB
    client = chromadb.PersistentClient(path=CHROMA_DB_PATH)
    vectorstore = Chroma(client=client, collection_name=COLLECTION_NAME, embedding_function=embeddings)

    # قراءة الملف بالدفعات لتجنب امتلاء الذاكرة (RAM)
    reader = pd.read_csv(OUT_CSV, chunksize=500)
    for i, chunk in enumerate(reader):
        chunk = chunk.fillna('')
        texts = chunk['text'].astype(str).tolist()
        metadatas = chunk[['source_pdf']].to_dict('records')
        vectorstore.add_texts(texts=texts, metadatas=metadatas)
        print(f"✅ تم حفظ الدفعة {i+1} (إجمالي السجلات: {(i+1)*500})")

    return vectorstore

# 4. تشغيل بناء/تحميل القاعدة
try:
    vectorstore = get_vectorstore()

    # 5. إعداد Gemini (حل مشكلة 404 عبر تحديد api_version)
    print("⏳ جارٍ الاتصال بمحرك Gemini...")
    llm = ChatGoogleGenerativeAI(
        model="gemini-1.5-flash",
        temperature=0.1,
        client_options={"api_version": "v1"} # هذا السطر يحل مشكلة 404 نهائياً
    )

    # 6. بناء محرك الإجابة (RAG Chain)
    retriever = vectorstore.as_retriever(search_kwargs={"k": 3})
    template = """أنت خبير متخصص في اضطراب طيف التوحد. أجب بدقة وهدوء بناءً على السياق المرفق فقط.
    إذا لم تجد الإجابة، أخبر المستخدم أن المعلومة غير متوفرة في الملفات المرفقة حالياً.

    السياق:
    {context}

    سؤال المستخدم: {question}

    الإجابة:"""

    prompt = ChatPromptTemplate.from_template(template)

    def format_docs(docs):
        return "\n\n---\n\n".join(doc.page_content for doc in docs)

    rag_chain = (
        {"context": retriever | format_docs, "question": RunnablePassthrough()}
        | prompt
        | llm
    )

    print("\n🎉 النظام جاهز تماماً!")

    # 7. واجهة المحادثة البسيطة
    user_input = input("\nاسألني أي شيء عن التوحد (أو اكتب 'خروج'): ")
    if user_input.lower() not in ['خروج', 'exit', 'quit']:
        print("\n⏳ جارٍ البحث عن الإجابة...")
        response = rag_chain.invoke(user_input)
        print(f"\nالإجابة:\n{response.content}")

except Exception as e:
    print(f"\n❌ حدث خطأ أثناء التشغيل: {e}")

📘 Notebook Section 3

Persistent Vector Store Loading & Inference

This section loads a previously built and persisted vector database from disk (Google Drive) and performs retrieval-augmented inference without re-computing embeddings, enabling fast and cost-efficient chatbot execution.

In [ ]:
import os
import google.generativeai as genai
from google.colab import drive
from langchain_chroma import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings

# 1. ربط Google Drive وتحميل البيانات
drive.mount('/content/drive')
DRIVE_SAVE_PATH = "/content/drive/MyDrive/autism_rag_metadata"

# 2. إعداد Gemini (المفتاح الذي قدمته سابقاً)
# تأكد من عدم وجود مسافات في المفتاح
API_KEY = "put your API_KEY here"
genai.configure(api_key=API_KEY)

# 3. تحميل نموذج التضمين وقاعدة البيانات (الـ 1500 مقطع)
print("⏳ جارٍ استعادة الـ 1500 مقطع من Google Drive...")
embeddings = HuggingFaceEmbeddings(model_name='intfloat/multilingual-e5-large')
vectorstore = Chroma(
    persist_directory=DRIVE_SAVE_PATH,
    embedding_function=embeddings,
    collection_name="autism_collection"
)
print("✅ قاعدة البيانات جاهزة.")

# 4. دالة الإجابة النهائية (تستخدم الموديل المستقر gemini-1.5-flash)
def ask_gemini_direct(query):
    try:
        # البحث في الـ 1500 مقطع
        docs = vectorstore.similarity_search(query, k=3)
        context = "\n\n".join([d.page_content for d in docs])

        # استخدام الموديل مباشرة لتجنب أخطاء 404
        # ملاحظة: إذا فشل 1.5-flash، استخدم 'gemini-pro'
        model = genai.GenerativeModel('gemini-1.5-flash')

        full_prompt = f"""أنت خبير في التوحد. أجب بناءً على المعلومات المرفقة فقط.

        المعلومات المستخرجة من ملفاتك:
        {context}

        سؤال المستخدم: {query}

        الإجابة:"""

        response = model.generate_content(full_prompt)
        return response.text
    except Exception as e:
        # إذا حدث خطأ 404، نحاول استخدام النسخة الأقدم والأكثر توافقاً
        if "404" in str(e):
            model_backup = genai.GenerativeModel('gemini-pro')
            response = model_backup.generate_content(full_prompt)
            return response.text
        return f"❌ حدث خطأ: {e}"

# 5. تشغيل المحادثة
print("-" * 30)
user_input = input("اسألني عن التوحد من واقع ملفاتك: ")
if user_input:
    print("⏳ جارٍ تحليل المعلومات وتوليد الإجابة...")
    result = ask_gemini_direct(user_input)
    print(f"\nالإجابة:\n{result}")

📘 Notebook Section 4

RAG Chatbot Runtime with GPT-4o Mini

This section performs real-time retrieval-augmented generation using a persisted vector database and GPT-4o-mini.
It retrieves the most relevant chunks from your database, assembles them as context, and generates precise answers grounded in your documents.

Uses LangChain abstractions (retriever | prompt | llm)

Shows snippets for transparency

Guards against hallucination by instructing the model to answer based only on the context

In [2]:
# 1. تثبيت المكتبات (للتأكد من توافق الإصدارات)
print("⏳ جارٍ التأكد من المكتبات...")
!pip install -U langchain-openai langchain-chroma langchain-community chromadb sentence-transformers --quiet

import os
from google.colab import drive, userdata
from langchain_chroma import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

# 2. ربط Google Drive
if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

# 3. إعداد المفاتيح والمسارات
try:
    os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
except:
    # إذا لم تكن تستخدم Secrets، ضع مفتاحك هنا
    os.environ["OPENAI_API_KEY"] = "put your OPENAI_API_KEY here"

DRIVE_SAVE_PATH = "/content/drive/MyDrive/autism_rag_metadata"

# 4. تحميل التضمينات وقاعدة البيانات
print("📂 جارٍ تحميل قاعدة البيانات...")
embeddings = HuggingFaceEmbeddings(model_name='intfloat/multilingual-e5-large')

vectorstore = Chroma(
    persist_directory=DRIVE_SAVE_PATH,
    embedding_function=embeddings,
    collection_name="autism_collection"
)

# 5. إعداد الموديل
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.1)

# 6. بناء محرك البحث (Retriever)
# استخدمنا k=5 لزيادة كمية المعلومات المستخرجة
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

template = """أنت خبير متخصص في اضطراب طيف التوحد. أجب بناءً على السياق المرفق فقط.

السياق:
{context}

السؤال: {question}

الإجابة:"""

prompt = ChatPromptTemplate.from_template(template)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
)

# 7. الدالة المصححة (تم تغيير get_relevant_documents إلى invoke)
def ask_and_verify(question):
    print(f"\n🔍 السؤال: {question}")

    # التعديل هنا: استخدام invoke بدلاً من get_relevant_documents
    try:
        relevant_docs = retriever.invoke(question)

        print("\n--------------------------------------------------")
        print(f"📋 تم العثور على {len(relevant_docs)} مقاطع ذات صلة في ملفاتك:")
        for i, doc in enumerate(relevant_docs):
            snippet = doc.page_content[:150].replace('\n', ' ')
            print(f"[{i+1}] {snippet}...")
        print("--------------------------------------------------\n")

        # توليد الإجابة
        print("🤖 إجابة GPT-4o-mini:")
        response = rag_chain.invoke(question)
        print(response.content)

    except Exception as e:
        print(f"❌ حدث خطأ: {e}")

# --- تشغيل السؤال ---
ask_and_verify("ما هي النصائح العملية للتعامل مع نوبات الغضب؟")

⏳ جارٍ التأكد من المكتبات...
📂 جارٍ تحميل قاعدة البيانات...

🔍 السؤال: ما هي النصائح العملية للتعامل مع نوبات الغضب؟

--------------------------------------------------
📋 تم العثور على 5 مقاطع ذات صلة في ملفاتك:
[1] عم األكاديمي والبيئي واالجتماعي والحسي المناسب والتعديل على البيئة والتوقعات هي الطريقة األكثر فعالية. يوجد العديد من االستراتيجيات التي يمكن استخدامه...
[2] لوسع: 5 نصائح للوالدين 1. تعلم أن تكون أفضلمناصر يمكنك أن تكون عليه لطفلك. كن واسع االطالع. استفد من جميع الخدمات المتاحة لك في مجتمعك. سوف تقابل المم...
[3] زداد نوبات الغضب لدى األطفال الذين يعانون من آالم .المشكلة السلوكية وعالجها غي األسنان أو المعدة. كما قد تُالحظ العصبية لدى األطفال الذين يعانون من حس...
[4] ادئة تناول وجبة خفيفة ارتداء سترة مثقلة مضغ العلكة أو تناول المشروبات بالمصاصات المصنوعة من الفينيل أو األطعمة المتفتتة أو المطاطية إبقاء اإلضاءة خافت...
[5] ول الحصول على قسط من الراحة. إذا كنت تنام بشكل منتظم، فستكون مستعدً ا بشكل أفضل التخاذ قرارات جيدة، وكن صبورً ا أكثر مع طفلك وأكثر قدرة على التعامل مع..